# Datenvorbereitung
Zunächst werden für alle gewünschten Einflussfaktoren die Daten beschafft und in ein CSV-Format mit lat/lon oder GeoJSON-Geoemtry gebracht, sodass zu allen Adressen die gewünschten Eigenschaften vorliegen.

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from shapely.geometry import box

from helper import load_geocsv, s

EPSG_4326 = "EPSG:4326"

# BBOX für Brandenburg an der Havel
CITY_BOUNDING_BOX = box(12.3120236786, 52.2938432979, 12.7562682548, 52.5594777244)

#  Stadtzentrum für Brandenburg an der Havel
CITY_CENTER = (52.4116351153561, 12.556331280534392) # Jahrtausendbrücke
#CITY_CENTER = 52.40905351242835, 12.517490967204653 # Visuelles Zentrum, damit periphere Cluster sichtbar bleiben

## Adressen einlesen
Alle Adressen des Zielgebiets als CSV in einen GeoDataFrame einlesen.

In [ ]:
gdf = load_geocsv("out/adressen_mit_zentrum_routen.csv")

# Adressen ohne Geometrie entfernen
gdf = gdf[~gdf.geometry.isna()].copy()

# Variante nur für urbanes Zentrum: Filter auf Adressen mit Distanz < 7500 m
# Auskommentieren für ganzen Datensatz!
# gdf = gdf.loc[gdf["distance_m"] < 7500].copy()

## Explorative Datenanalyse

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns           # nur für schönere Plots
from helper import load_geocsv, make_merge_addr

# Eindeutigen Merge-baren String aus Adresse erzeugen
gdf["Adresse_merge"] = gdf.apply(make_merge_addr, axis=1)

gdf = gdf.rename(columns={"geojson": "center_route",
                                    "distance_m" : "center_distance"})

# Ein paar unbenötigte Spalten entfernen
gdf = gdf.drop(columns=["index_right", "duration_s", "display_name", "type", "category", "Adresse_query"], errors="ignore")

# Histogramm der Distanzen zum Zentrum
plt.figure(figsize=(6,3))
sns.histplot(gdf["center_distance"], bins=60, kde=False)
plt.title("Verteilung der Fußentfernung zum Zentrum")
plt.xlabel("Distanz (Meter)")
plt.ylabel("Häufigkeit")
plt.show()
print(gdf.shape)

## Daten bereinigen

In [ ]:
print(gdf.shape)
# Dubletten aus Adressen entfernen
dups = gdf["Adresse_merge"].value_counts()
dups = dups[dups > 1]
print(f"{len(dups)} doppelte Adressen gefunden")
print(dups.head())

# Ersten Treffer behalten
gdf = gdf.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")
print(gdf.shape)


## Einzelhandel
Separates Skript ```einzelhandel-adressen.py``` ausfühen, um Datei "adressen_mit_einzelhandel.csv" zu erzeugen.
Durch fußläufiges Routing berechnete Faktoren:
- Anzahl von Einkaufsmöglichkeiten im Umkreis von 500 m (Anforderung AK MSP: mind. 1)
- Geringste Distanz zum nächsten Einzelhandel

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

# Lade vorberechnete Einzelhandel-Faktoren
gdf_retail = load_geocsv("out/adressen_mit_einzelhandel_routen.csv")

# Merge mit Haupt-GDF
gdf_retail["Adresse_merge"] = gdf_retail.apply(make_merge_addr, axis=1)
gdf_retail = gdf_retail.sort_values("Adresse_merge").drop_duplicates("Adresse_merge", keep="first")

dups = gdf_retail["Adresse_merge"].value_counts()
print(dups[dups > 1])

einzelhandel_attribute = ["einzelhandel_route","einzelhandel_min_distance", "einzelhandel_500m_count"]

gdf = pd.merge(
    gdf,
    gdf_retail[["Adresse_merge"] + einzelhandel_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)
print(gdf.shape)
print(gdf[["Adresse_merge", "einzelhandel_min_distance"]].head())

print(gdf.shape)

# Speicher freigeben
del(gdf_retail)

In [ ]:
# Verteilung Distanz zum nächsten Markt
print(gdf.columns)
sns.histplot(gdf["einzelhandel_min_distance"].dropna(), bins=40, kde=False)
plt.title("Distanz zum nächsten Lebensmittelmarkt")
plt.xlabel("Meter Fußweg"); plt.ylabel("Adressen")
plt.show()

# Scatter Zentralität vs. Nahversorgung
sns.scatterplot(x="center_distance", y="einzelhandel_min_distance", data=gdf, alpha=.3)
plt.xlabel("Distanz Zentrum (m)"); plt.ylabel("Distanz nächster Markt (m)")
plt.show()

## Lärmbelastung
Siehe ```laerm.ipynb``` zur Erzeugung, ansonsten einfach ```data/adressen_mit_laerm.csv``` verwenden und per Spalte 'geometry' verschneiden.

Definition Lärm-Index:
- NaN / leer: kein gemessener Straßenlärm
- 0: 55 - 59 dB
- 1: 60 - 64 dB
- 2: 65 - 69 dB
- 3: 70 - 74 dB
- 4: >= 75 db

In [ ]:
gdf_laerm_karte = load_geocsv("out/adressen_mit_laerm.csv")
print(gdf_laerm_karte.shape)
print(gdf.shape)

# Merge des Lärmindex mit Haupt-GDF
gdf_laerm_karte["Adresse_merge"] = gdf_laerm_karte.apply(make_merge_addr, axis=1)
gdf_laerm_karte["laerm_index_tag"] = gdf_laerm_karte["Laerm_index_tag"].fillna(-0.1) # lowercase

# Deduplicate, im Zweifel nimm den lautesten Wert
gdf_laerm_karte = (
    gdf_laerm_karte
    .sort_values("laerm_index_tag", ascending=False)
    .drop_duplicates("Adresse_merge", keep="first")
)

laerm_attribute = ["laerm_index_tag"]
gdf = pd.merge(
    gdf,
    gdf_laerm_karte[["Adresse_merge"] + laerm_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_many"
)

#print(gdf.columns)
print(gdf.shape)

## Nähe zu ÖPNV-Haltestellen

Die Qualität des ÖPNV wird anhand der Fußläufigkeit zur nächsten Haltestelle definiert. Die Häufigkeit von Abfahrten (Headway) wurde zwar berechnet, ist aber in der aktuellen Form noch kein verlässlicher Indikator. Die Daten dafür stammen vom Verkehrsverbund Berlin-Brandenburg (VBB), Lizenz: CC BY 4.0, [zu den Daten](https://unternehmen.vbb.de/digitale-services/datensaetze) bzw. der Stadt Brandenburg an der Havel ("2026_Haltestellen.csv").

Vorausgesetzte Datensätze:
- ```data/GTFS/stops.txt```, der die Haltestellen mit ihren Geokoordinaten enthält.
- ```data/adressen_mit_haltestellen_routen.csv```, der die Routen zu den Haltestellen und Anzahl von Haltestellen im Radius 500m und 800m enthält.
- ```2026_Haltestellen.csv```, der nur die Haltestellen mit Geokoordinaten enthält.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load GTFS stops data
stops = pd.read_csv("data/GTFS/stops.txt")
stops["geometry"] = stops.apply(lambda row: Point(row["stop_lon"], row["stop_lat"]), axis=1)
gdf_stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=EPSG_4326)

# Convert to UTM for precise metric distance calculations
gdf = gdf.to_crs(epsg=32633)
gdf_stops = gdf_stops.to_crs(epsg=32633)

# Assign nearest stop to each address based on Point(X, Y) geometry
gdf["nearest_stop_id"] = gdf.geometry.apply(
    lambda pt: gdf_stops.loc[gdf_stops.distance(pt).idxmin(), "stop_id"]
)

# Read routes from adresses to stops and stop count from prepared CSV
gdf_haltestellen = load_geocsv("out/adressen_mit_haltestellen_routen.csv")
gdf_haltestellen["Adresse_merge"] = gdf_haltestellen.apply(make_merge_addr, axis=1)
gdf_haltestellen = gdf_haltestellen.drop_duplicates("Adresse_merge").copy()

print("gdf_haltestellen shape:", gdf_haltestellen.shape)
print("gdf shape (vor Merge):", gdf.shape)

# Merge haltestellen attributes into main GeoDataFrame
haltestellen_attribute = ["haltestellen_route", "haltestellen_min_distance", "haltestellen_count_within_500m", "haltestellen_count_within_800m"]
gdf = pd.merge(
    gdf,
    gdf_haltestellen[["Adresse_merge"] + haltestellen_attribute],
    on="Adresse_merge",
    how="left",
    validate="one_to_one"
)

print("gdf shape (nach Merge):", gdf.shape)

## Begrenzung durch dauerhafte Barrieren
Als zusätzlicher Indikator wird berechnet, ob eine Adresse durch eine Bahnlinie vom Stadtzentrum getrennt ist. Damit wird abgebildet, ob eine Adresse trotz Nähe zum Zentrum durch eine Barriere erschwerten Zugang hat (z. B., durch Wartezeiten an Bahnübergängen).

Die Variable z_rail_penalty modelliert im Modell später die Barrierewirkung von Bahnlinien als festen, moderaten Mobilitätsabschlag für Adressen, die vom Zentrum aus gesehen hinter einer Bahnlinie liegen.

In [ ]:
import osmnx as ox
from shapely.geometry import LineString
# Schritt 1: Bahnlinien von OSM laden
place = "Brandenburg an der Havel, Germany"

# Eisenbahnlinien holen
rails = ox.features_from_place(
    place,
    tags={"railway": True}  # includes rail, light_rail, tram, etc.
)

# nur Schienenverkehr (keine Haltestellen)
rails = rails[rails["railway"].isin(["rail"])]
gdf.crs = EPSG_4326
rails.crs = EPSG_4326

# Schritt 2: Fußĺäufige Routen zum Stadtzentrum auf Schnittpunkt mit Bahnlinie prüfen und im Datensatz speichern
def route_crosses_rail(route_geojson_str, rail_geom):
    if not isinstance(route_geojson_str, str):
        return False

    try:
        geo = json.loads(route_geojson_str)
    except:
        return False

    if geo.get("type") != "LineString":
        return False

    # Koordinaten (lon, lat) -> Shapely LineString nutzen
    coords = geo["coordinates"]
    line = LineString(coords)

    # Schnitt-Test
    return line.intersects(rail_geom)

rail_union = rails.union_all()
gdf["behind_rail_from_center"] = gdf["center_route"].apply(
    lambda r: int(route_crosses_rail(r, rail_union))
)

print(gdf.shape)

In [ ]:
import json
import folium

# 10 zufällige Adressen ziehen
sample_gdf = gdf.sample(10, random_state=42)

center_lat, center_lon = CITY_CENTER  # CITY_CENTER ist [lat, lon]
center_point = (center_lat, center_lon)
m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Zentrum markieren
folium.Marker(
    location=CITY_CENTER,
    tooltip="Stadtzentrum",
    icon=folium.Icon(color="red", icon="star")
).add_to(m)

# Route + Marker je Adresse
for idx, row in sample_gdf.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        continue

    addr_point = (row["lat"], row["lon"])
    behind_flag = row.get("behind_rail_from_center", None)

    tooltip = f"Adresse {idx}<br>behind_rail_from_center: {behind_flag}"

    # Route zum Zentrum (GeoJSON)
    route_json = row.get("center_route")

    if isinstance(route_json, str):
        try:
            route_geo = json.loads(route_json)

            if route_geo.get("type") == "LineString":
                # GeoJSON Koordinaten sind (lon, lat)
                coords = [(c[1], c[0]) for c in route_geo["coordinates"]]

                folium.PolyLine(
                    locations=coords,
                    color="blue",
                    weight=4,
                    opacity=0.7,
                    tooltip=f"Route zu Adresse {idx}"
                ).add_to(m)

        except Exception as e:
            print(f"Fehler beim Zeichnen der Route für Adresse {idx}: {e}")

    # Adressmarker
    folium.CircleMarker(
        location=addr_point,
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.8,
        tooltip=folium.Tooltip(tooltip)
    ).add_to(m)

# Bahnlinien-Layer
bahn_layer = folium.FeatureGroup(name="Bahnlinien (OSM)", show=True)

for _, row in rails.iterrows():
    geom = row.geometry
    if geom is None:
        continue

    if geom.geom_type == "LineString":
        coords = [(y, x) for x, y in geom.coords]
        folium.PolyLine(
            locations=coords,
            color="darkred",
            weight=4,
            opacity=0.8
        ).add_to(bahn_layer)

    elif geom.geom_type == "MultiLineString":
        for part in geom:
            coords = [(y, x) for x, y in part.coords]
            folium.PolyLine(
                locations=coords,
                color="darkred",
                weight=4,
                opacity=0.8
            ).add_to(bahn_layer)

bahn_layer.add_to(m)

print(gdf.shape)
m

# Visualisierung zentraler Indikatoren
Alle Einflussfaktoren (Supermärkte, Ärzte, Schulen etc.) werden zur Plausibilitätsprüfung auf einer Karte visualisiert.

In [ ]:
import folium
import branca.colormap as cm
from helper import add_markers_from_csv, add_medcenter_markers, STRASSENNAME, HAUSNUMMER, HAUSNUMMERZUSATZ

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")


# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="out/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", layer_name="Einzelhandel")
add_markers_from_csv(map_obj=m, csv_path="out/haltestellen_geocoded.csv", color="lightgray", icon="bus", layer_name="Haltestellen")

# Lärmindex aus dem Geopackage
gdf_laerm_karte = gpd.read_file("data/laerm.gpkg", layer="laerm")
gdf_laerm_karte = gdf_laerm_karte.clip(CITY_BOUNDING_BOX)
value_column = "isov1"
min_val = gdf_laerm_karte[value_column].min()
max_val = gdf_laerm_karte[value_column].max()
colormap = cm.LinearColormap(colors=["green", "yellow", "red"], vmin=min_val, vmax=max_val)
colormap.caption = "Lärmpegel (LDEN in dB)"

# Alle Adressen als Punkte
adress_layer = folium.FeatureGroup(name="Wohnadressen", show=False)
for _, row in gdf.iterrows():
    if pd.notna(row["lat"]) and pd.notna(row["lon"]):
        strasse = s(row.get(STRASSENNAME))
        hsnr = s(row.get(HAUSNUMMER))
        hsnrzus = s(row.get(HAUSNUMMERZUSATZ))

        tooltip = strasse + " " + hsnr

        adressen_map = folium.CircleMarker(
            location=[row["lat"], row["lon"]],
            radius=3,
            color="violet",
            fill=True,
            fill_opacity=0.6,
            tooltip=tooltip
        ).add_to(adress_layer)
adress_layer.add_to(m)

def style_function(feature):
    value = feature["properties"][value_column]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.2
    }

laerm_layer = folium.FeatureGroup(name="Lärmkarte (LDEN 2022)")
folium.GeoJson(
    gdf_laerm_karte,
    style_function=style_function,
).add_to(laerm_layer)
laerm_layer.add_to(m)
colormap.add_to(m)

# Bahnlinien layer hinzufügen
bahn_layer = folium.FeatureGroup(name="Bahnlinien", show=True)

for _, row in rails.iterrows():
    geom = row.geometry
    if geom is None:
        continue

    if geom.geom_type == "LineString":
        coords = [(y, x) for x, y in geom.coords]
        folium.PolyLine(
            locations=coords,
            color="darkred",
            weight=4,
            opacity=0.8
        ).add_to(bahn_layer)

    elif geom.geom_type == "MultiLineString":
        for part in geom:
            coords = [(y, x) for x, y in part.coords]
            folium.PolyLine(
                locations=coords,
                color="darkred",
                weight=4,
                opacity=0.8
            ).add_to(bahn_layer)

bahn_layer.add_to(m)

# Wohnadressen nach "cluster_kmeans" farb-codiert oben drauf

# Schaltbare Layer
folium.LayerControl(collapsed=False).add_to(m)
del gdf_laerm_karte  # Speicher freigeben
print(gdf.shape)
m

# Scoring / Punktesystem
Hier werden z-Werte zu alle **Einflussfaktoren** gebildet, um die Abweichung einer Ausprägung vom Standard (im betrachteten Gebiet) zu erfassen. Weiterhin werden die **Gewichte** festgelegt, mit denen die Faktoren in die Wohnlagenbewertung eingehen.

In [ ]:
import numpy as np
from scipy.stats import zscore

# ---------------------------------------
# 1) Numerische & binäre Features
# ---------------------------------------
numeric_features = (
        ["center_distance"] +
        haltestellen_attribute +
        #headway_attribute +
        einzelhandel_attribute +
        laerm_attribute 
        )

#binary_features = ["behind_rail_from_center"]

# Masken nur auf numerische Daten!
mask_all = gdf[numeric_features].notna().all(axis=1)

# ---------------------------------------
# 2) Z-Scores für numerische Variablen
#    (immer: hoch = gut!)
# ---------------------------------------
def safe_z(x):
    z = zscore(x)
    return np.where(np.isfinite(z), z, 0)

# Zentralität
gdf.loc[mask_all, "z_centrality"] = -safe_z(gdf.loc[mask_all, "center_distance"])

# Einzelhandel
gdf.loc[mask_all, "z_einzelhandel_distance"]    = -safe_z(gdf.loc[mask_all, "einzelhandel_min_distance"])
gdf.loc[mask_all, "z_einzelhandel_near_500"]    =  safe_z(gdf.loc[mask_all, "einzelhandel_500m_count"])

# Lärm
gdf.loc[mask_all, "z_laerm_index_tag"]          = -safe_z(gdf.loc[mask_all, "laerm_index_tag"])
# Einseitiger Strafterm: nur überdurchschnittlicher Lärm führt zu Abzug
# Wichtig: Im Gesamt-Score geht Lärm nur ueber z_noise_penalty ein (kein Doppelcount)
gdf.loc[mask_all, "z_noise_penalty"]           = -0.30 * np.clip(safe_z(gdf.loc[mask_all, "laerm_index_tag"]), 0, None)

# Mobilität
gdf.loc[mask_all, "z_haltestelle_distance"]     = -safe_z(gdf.loc[mask_all, "haltestellen_min_distance"])
gdf.loc[mask_all, "z_haltestellen_count_within_500m"] =  safe_z(gdf.loc[mask_all, "haltestellen_count_within_500m"])
#gdf.loc[mask_all, "z_headway_score"]            = -safe_z(gdf.loc[mask_all, "headway_avg"]) # vorläufig nicht in der Bewertung
#gdf["z_rail_penalty"] = -0.3 * gdf["behind_rail_from_center"] # Leichte Strafe, wenn durch Bahnlinie vom Zentrum getrennt

# ---------------------------------------
# 3) Score-Dimensionen (alle Summen = 1.0)
# ---------------------------------------
### 3.1 Zentralität (1 Dimension, Summe = 1.0)
mask_mm_central = gdf["center_distance"].notna()
gdf.loc[mask_mm_central, "score_zentralitaet"] = gdf.loc[mask_mm_central, "z_centrality"]


### 3.2 Versorgung (Summe = 1.0)
mask_mm_versorgung = gdf[[
    "einzelhandel_min_distance",
    "einzelhandel_500m_count"
]].notna().all(axis=1)

gdf.loc[mask_mm_versorgung, "score_versorgung"] = (
        0.50 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_distance"]
        + 0.50 * gdf.loc[mask_mm_versorgung, "z_einzelhandel_near_500"]
)

### 3.3 Mobilität (Summe = 1.0)
mask_mm_mobilitaet = gdf[[
    "haltestellen_min_distance",
    "haltestellen_count_within_500m"
]].notna().all(axis=1)

gdf.loc[mask_mm_mobilitaet, "score_mobilitaet"] = (
        0.50 * gdf.loc[mask_mm_mobilitaet, "z_haltestelle_distance"]
       # + 0.40 * gdf.loc[mask_mm_mobilitaet, "z_rail_penalty"]
        + 0.50 * gdf.loc[mask_mm_mobilitaet, "z_haltestellen_count_within_500m"]
)


### 3.4 Umwelt (Summe = 1.0)
# Umwelt (Summe = 1.0)
mask_mm_umwelt = gdf[[
    "laerm_index_tag",
]].notna().all(axis=1)

gdf.loc[mask_mm_umwelt, "score_umwelt"] = (
        + 1.0 * gdf.loc[mask_mm_umwelt, "z_laerm_index_tag"]
)  # Diagnosewert; score_total nutzt stattdessen z_noise_penalty (kein Doppelcount)

# ---------------------------------------
# 4) Gesamt-Score (alle Dimensionen)
# ---------------------------------------
score_all_vars = [
    "score_zentralitaet",
    "score_versorgung",
    "score_mobilitaet"
]
#### ANPASSUNG GEWICHTE HIER: ####

mask_all_scores = gdf[score_all_vars + ["z_noise_penalty"]].notna().all(axis=1)
gdf.loc[mask_all_scores, "score_total"] = (
        (1.0 / 3.0) * gdf.loc[mask_all_scores, "score_zentralitaet"]
        + (1.0 / 3.0) * gdf.loc[mask_all_scores, "score_versorgung"]
        + (1.0 / 3.0) * gdf.loc[mask_all_scores, "score_mobilitaet"]
        + gdf.loc[mask_all_scores, "z_noise_penalty"]  # separater Laerm-Malus
)

print("Anzahl gültiger Gesamt-Scores:", gdf["score_total"].notna().sum())
gdf[[
    "score_total",
    "score_zentralitaet",
    "score_versorgung",
    "score_umwelt",
    "score_mobilitaet",
]].corr()

# Validierung

Im Folgenden werden die Z-Variablen genutzt, um mittels K-Means-Clustering Wohnlagen zu identifizieren. Zunächst wird die optimale Clusteranzahl mittels Elbow-Methode und Silhouetten-Analyse bestimmt. Danach wird das finale K-Means-Modell mit der gewählten Clusteranzahl trainiert und die Wohnlagen den Adressen zugewiesen.

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Z-Variablen aus allen Kategorien
z_vars = [
    # Zentralität
    "z_centrality",

    # Einzelhandel
    "z_einzelhandel_distance",
    "z_einzelhandel_near_500",

    # Lärm
    "z_laerm_index_tag",

    # Haltestellen / Mobilität
    "z_haltestelle_distance",
    "z_haltestellen_count_within_500m",
    #"z_headway_score",
    #"z_rail_penalty",   # bewusst KEIN zscore, sondern Strafterm
    ]

# Validierung
missing = [c for c in z_vars if c not in gdf.columns]
if missing:
    raise ValueError(f"Diese Z-Variablen fehlen im gdf: {missing}")

# Datenmatrix: nur vollständige Zeilen
X = gdf[z_vars].dropna().values

# Elbow-Methode
inertia = []
cluster_range = range(2, 15)

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)
    inertia.append(model.inertia_)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, inertia, marker='o')
plt.title("Elbow-Methode: KMeans-Inertia vs. Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Inertia (Distanz innerhalb der Cluster)")
plt.xticks(cluster_range)
plt.grid(True)
plt.show()

In [ ]:
from pathlib import Path
# Silhouetten-Analyse
from sklearn.metrics import silhouette_score

silhouettes = []

for k in cluster_range:
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels, random_state=42)
    silhouettes.append(score)

plt.figure(figsize=(8,5))
plt.plot(cluster_range, silhouettes, marker='o', color="green")
plt.title("Silhouetten-Score pro Clusteranzahl")
plt.xlabel("Anzahl Cluster (k)")
plt.ylabel("Durchschn. Silhouetten-Koeffizient")
plt.xticks(cluster_range)
plt.grid(True)

# Zielordner anlegen (falls nicht vorhanden)
out_dir = Path("plots")
out_dir.mkdir(parents=True, exist_ok=True)

# Bild speichern
plt.savefig(
    out_dir / "silhouette_scores.png",
    dpi=300,
    bbox_inches="tight"
)

plt.show()


In [ ]:
NUMBER_OF_CLUSTERS = 6

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

k = NUMBER_OF_CLUSTERS
seeds = [0, 1, 2, 3, 4, 5, 42, 123]

results = []

for seed in seeds:
    km = KMeans(
        n_clusters=k,
        init="k-means++",
        random_state=seed,
        n_init="auto",
        max_iter=300,
    ).fit(X)
    labels = km.labels_
    inertia = km.inertia_
    sil = silhouette_score(X, labels)
    results.append((seed, inertia, sil))

for seed, inertia, sil in results:
    print(f"seed={seed}, inertia={inertia:.0f}, silhouette={sil:.3f}")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# 1) KMeans
X = gdf[z_vars].dropna().values  # vollständige Zeilen
model = KMeans(n_clusters=NUMBER_OF_CLUSTERS, random_state=42).fit(X)

mask_complete = gdf[z_vars].notna().all(axis=1)
gdf.loc[mask_complete, "cluster_kmeans"] = model.labels_

cluster_centers = pd.DataFrame(model.cluster_centers_, columns=z_vars)
cluster_centers.index.name = "Cluster"
print(cluster_centers.round(2))

# 2) PCA (3 Komponenten)
pca3 = PCA(n_components=3, random_state=42)

X_pca3 = pca3.fit_transform(X)
centers_pca3 = pca3.transform(cluster_centers.to_numpy())
print("Explained variance ratio (3 PCs):", pca3.explained_variance_ratio_)

# Datenframe für Punkte
df_pca = pd.DataFrame({
    "PC1": X_pca3[:, 0],
    "PC2": X_pca3[:, 1],
    "PC3": X_pca3[:, 2],
    "cluster_kmeans": model.labels_
})

# Datenframe für Cluster-Zentren
df_centers = pd.DataFrame({
    "PC1": centers_pca3[:, 0],
    "PC2": centers_pca3[:, 1],
    "PC3": centers_pca3[:, 2],
    "cluster_kmeans": range(NUMBER_OF_CLUSTERS)
})

# 3) Interaktive 3D-Plotly-Grafik
colors = px.colors.qualitative.Dark24  # 24 Farben → genug Reserve
fig = go.Figure()

# Clusterpunkte einzeichnen
for cl in sorted(df_pca["cluster_kmeans"].unique()):
    sub = df_pca[df_pca["cluster_kmeans"] == cl]
    fig.add_trace(go.Scatter3d(
        x=sub["PC1"],
        y=sub["PC2"],
        z=sub["PC3"],
        mode="markers",
        marker=dict(
            size=3.5,
            color=colors[cl % len(colors)],
            opacity=0.65
        ),
        name=f"Cluster {cl}"
    ))

# Clusterzentren
fig.add_trace(go.Scatter3d(
    x=df_centers["PC1"],
    y=df_centers["PC2"],
    z=df_centers["PC3"],
    mode="markers",
    marker=dict(
        size=6,
        color="black",
        symbol="x",
        opacity=0.9
    ),
    name="Cluster-Zentren"
))

# Layout
fig.update_layout(
    title="Hauptkomponentenprojektion (PCA) – KMeans-Cluster",
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    ),
    height=750,
    legend=dict(itemsizing="constant")
)

fig.show()


In [ ]:
import folium
import matplotlib.colors as mcolors

mask = gdf["lat"].notna() & gdf["lon"].notna() & gdf["cluster_kmeans"].notna()
gdf = gdf.loc[mask].copy()
gdf["cluster_kmeans"] = gdf["cluster_kmeans"].astype(int)

def get_cluster_colors(n_clusters):
    cmap = plt.get_cmap("tab20")   # 20 unterscheidbare Farben
    colors = {
        i: mcolors.to_hex(cmap(i % 20))
        for i in range(n_clusters)
    }
    return colors

cluster_colors = get_cluster_colors(NUMBER_OF_CLUSTERS)

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Add layer markers
add_markers_from_csv(map_obj=m, csv_path="out/einzelhandel_geocoded.csv", color="blue", icon="shopping-cart", layer_name="Einzelhandel")
add_markers_from_csv(map_obj=m,csv_path="out/haltestellen_geocoded.csv",color="lightgray", icon="bus", layer_name="Haltestellen")


for _, r in gdf.iterrows():
    c = cluster_colors.get(r["cluster_kmeans"], "#666666")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.85,
        tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r["cluster_kmeans"]}"
    ).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Nur Spalten verwenden, die im gdf enthalten sind
cols = [col for col in z_vars if col in gdf.columns]

# Korrelationsmatrix berechnen
corr = gdf[cols].corr()

# Plot
plt.figure(figsize=(20, 16))
sns.heatmap(corr, annot=True, cmap="vlag", center=0)
plt.title("Korrelationsmatrix der Z-Scores")
plt.tight_layout()
plt.show()


Die Korrelationsanalyse zeigt, dass .....


## Interaktive Karte zur Bewertung einzelner Adressen

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO

# Widget für Adresseingabe
text_input = widgets.Text(
    value='Kurstraße 15',
    placeholder='Straßenname Hausnummer',
    description='Adresse:',
    disabled=False
)

button = widgets.Button(description="Zeige Routen")
output = widgets.Output()

# Hilfsfunktion
def load_geojson(geo):
    """Konvertiert GeoJSON-Felder sicher in ein Python-Dict."""
    if geo is None:
        return None
    if isinstance(geo, float):  # NaN
        return None
    if isinstance(geo, dict):   # bereits dict
        return geo
    if isinstance(geo, str) and geo.strip() == "":
        return None
    if isinstance(geo, str):
        try:
            return json.loads(geo)
        except Exception:
            return None
    return None

def scale_score(z):
    """Skaliert Z-Score linear auf 0 - 10 für Bürgerverständlichkeit."""
    if pd.isna(z):
        return None
    score = (z + 3) / 6 * 10
    return max(0, min(10, round(score, 1)))

gdf["score_total_scaled"]        = gdf["score_total"].apply(scale_score)
gdf["score_zentralitaet_scaled"] = gdf["score_zentralitaet"].apply(scale_score)
gdf["score_versorgung_scaled"]   = gdf["score_versorgung"].apply(scale_score)
gdf["score_mobilitaet_scaled"]   = gdf["score_mobilitaet"].apply(scale_score)
gdf["score_umwelt_scaled"]       = gdf["score_umwelt"].apply(scale_score)

def score_color(value):
    if value is None:
        return "gray"
    if value >= 8:
        return "#2ecc71"   # grün
    if value >= 5:
        return "#f1c40f"   # gelb
    return "#e74c3c"        # rot

# Funktion zum Einfügen einer Route + Zielmarker
def add_route(m, geojson_raw, color, label, icon, distance=None):
    geo = load_geojson(geojson_raw)
    if not geo:
        return  # keine Route vorhanden

    if geo.get("type") == "LineString":
        try:
            coords = [(y, x) for x, y in geo["coordinates"]]
        except Exception as e:
            print(f"Fehler beim Lesen der Koordinaten für {label}: {e}")
            return

        distance_text = f" – {int(distance)} m" if distance else ""
        tooltip_text = f"{label}{distance_text}"

        folium.PolyLine(
            locations=coords,
            color=color,
            weight=4,
            opacity=0.9,
            tooltip=tooltip_text
        ).add_to(m)

        # Zielpunkt markieren
        end = coords[-1]
        folium.Marker(
            location=end,
            icon=folium.Icon(color=color, icon=icon, prefix="fa"),
            tooltip=f"Ziel: {label}{distance_text}"
        ).add_to(m)


# Funktion zum Anzeigen der Karte
def show_routes(_=None):
    output.clear_output(wait=True)
    with output:
        addr = text_input.value.strip().lower()

        filtered = gdf[gdf["Adresse_merge"].str.lower().str.contains(addr)].copy()
        if filtered.empty:
            print("Keine passende Adresse gefunden.")
            return

        row = filtered.iloc[0]

        m = folium.Map(
            tiles="cartodbpositron",
            location=CITY_CENTER,
            zoom_start=12,
        )

        # Farbige Score-Badges
        def badge(label, val):
            col = score_color(val)
            return f"<div style='padding:4px 8px;margin:2px;background:{col};color:white;border-radius:4px;display:inline-block;'>{label}: {val}</div>"

        building_type = get_building_type(row, building_type_cols)

        popup_html = f"""
        <div style="width:420px;font-family:Arial, sans-serif;">
          <h2>{row['Straßenname']} {row['Hsnr']}</h2>

          <h3>Gesamtbewertung</h3>
          {badge("Gesamt", row.get('score_total_scaled'))}

          <h3>Teilbereiche</h3>
          {badge("Zentralität", row.get('score_zentralitaet_scaled'))}
          {badge("Versorgung", row.get('score_versorgung_scaled'))}
          {badge("Mobilität", row.get('score_mobilitaet_scaled'))}
          {badge("Umwelt", row.get('score_umwelt_scaled'))}

          <h3 style="margin-top:15px;">Wohnlagen-Typ</h3>
          <ul>
            <li><b>Baualtersklasse:</b> {row.get("building_age_class", "unbekannt")}</li>
            <li><b>Gebäudetyp:</b> {building_type}</li>
          </ul>

          <h3 style="margin-top:15px;">Fußläufige Entfernungen</h3>
          <ul>
            <li><b>Entfernung Zentrum:</b> {int(row.get("center_distance",0))} m</li>
            <li><b>Nächste Haltestelle:</b> {int(row.get("haltestellen_min_distance",0))} m</li>
            <li><b>Nächster Einzelhandel:</b> {int(row.get("einzelhandel_min_distance",0))} m</li>
          </ul>
        </div>
        """

        popup = folium.Popup(popup_html, max_width=450)

        marker = folium.Marker(
            location=[row.lat, row.lon],
            popup=popup,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

        # Popup automatisch öffnen
        marker.add_child(folium.Popup(popup_html, max_width=450))
        marker.add_child(folium.map.Tooltip(""))  # kein Tooltip

        # Popup sofort anzeigen
        marker._popup = popup

        # Startmarker
        folium.Marker(
            location=[row.lat, row.lon],
            popup=popup_html,
        ).add_to(m)

        # ▸ Routen einfügen
        add_route(m, row.get("center_route"), "lightgray", "Weg zum Zentrum", "arrows-to-circle", row.get("center_distance"))
        add_route(m, row.get("haltestellen_route"), "beige", "Nächste Haltestelle", "bus", row.get("haltestellen_min_distance"))
        add_route(m, row.get("einzelhandel_route"), "purple", "Nächster Einzelhandel", "shop", row.get("einzelhandel_min_distance"))

        display(m)

# Button-Event
button.on_click(show_routes)

# UI anzeigen
display(text_input, button, output)


# Räumliches Clustering
Ein Ziel der Analyse ist die Entwicklung zusammenhängender Gebiete, die einer gemeinsamen Wohnlage zugeordnet werden können. Damit sollen "Insellagen", also mehrere abgeschnittene Bereiche mit derselben Wohnlage vermieden werden. Dafür wenden wir im Folgenden eine Glättung mit dem SKATER-Ansatz an (vgl. [Assunção et al. 2006](https://doi.org/10.1080/13658810600665111])).

In [ ]:
from libpysal import weights
from libpysal.weights import KNN
from spopt.region import Skater
from libpysal.weights import DistanceBand

gdf = gdf.set_crs(epsg=32633, allow_override=True)

# Index zurücksetzen
gdf = gdf.reset_index(drop=True)

z_vars = [
    'z_centrality',
    'z_einzelhandel_distance',
    'z_laerm_index_tag',
    'z_haltestelle_distance'
]

# Erzeuge Gewichtsmatrix
W = weights.contiguity.Queen.from_dataframe(gdf)
W.transform = "r" # Row-standardized, damit Gebiete mit vielen Nachbarn nicht übermäßig Einfluss haben

print("len(gdf):", len(gdf), "W.n:", W.n)
print("first ids:", W.id_order[:5])
print("n_components:", W.n_components, "islands:", len(W.islands))

sk = Skater(
    gdf, W, z_vars,
    n_clusters=NUMBER_OF_CLUSTERS,
    islands="ignore",
    floor=50 # Mindestanzahl Adressen je Region
)
sk.solve()

gdf["cluster_skater"] = sk.labels_

# Cluster-Karte

In [ ]:
import folium
import geopandas as gpd
import pandas as pd

# Daten laden & CRS vereinheitlichen
gdf_quartiere = gpd.read_file("data/Quartiere/2024_Quartiere.gpkg")
gdf_ortsteile = gpd.read_file("data/ortsteile_brandenburg.json", bbox=CITY_BOUNDING_BOX)
gdf_quartiere = gdf_quartiere.to_crs(4326)
gdf_ortsteile.drop(columns=["otl_aktualitaet"], inplace=True, errors="ignore") # Karte will keine Timestamps!
gdf = gdf.to_crs(4326)

assert gdf.crs.to_epsg() == 4326
assert gdf_quartiere.crs.to_epsg() == 4326
assert gdf_ortsteile.crs.to_epsg() == 4326

# Spatial Join: Adressen -> Quartiere
# alte Spalten sicher entfernen
gdf = gdf.drop(columns=["index_right", "mietkatego", "mietkategorie", "bezeichnun", "quartier"], errors="ignore")
cols_to_drop = [col for col in gdf.columns if col.endswith("_quartier")]

gdf = gdf.drop(columns=cols_to_drop, errors="ignore")
gdf = gpd.sjoin(
    gdf,
    gdf_quartiere[["bezeichnun", "mietkatego", "geometry"]],
    how="left",
    predicate="within",
)

# Spatial Join: Adressen -> Ortsteile
# alte Spalten sicher entfernen
gdf = gdf.drop(columns=["index_right", "otl_name", "index_left", "otl_name_left", "otl_name_right"], errors="ignore")
gdf = gpd.sjoin(
    gdf,
    gdf_ortsteile[["otl_name", "geometry"]],
    how="left",
    predicate="within",
)

gdf = gdf.drop(columns=["index_right"])

gdf = gdf.rename(columns={
    "otl_name": "ortsteil",
    "bezeichnun": "quartier",
    "mietkatego": "mietkategorie"
})

m = folium.Map(location=CITY_CENTER, zoom_start=13, tiles="cartodbpositron")

# Ortsteile layer
folium.GeoJson(
    gdf_ortsteile,
    name="Ortsteile",
    style_function=lambda feature: {
        'fillColor': '#1f78b4',
        'color': '#1f78b4',
        'weight': 1,
        'fillOpacity': 0.10,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["otl_name"],
        aliases=["Ortsteil"],
        localize=True,
        sticky=True
    )
).add_to(m)

#  Quartiere layer
folium.GeoJson(
    gdf_quartiere,
    name="Quartiere",
    style_function=lambda feature: {
        'fillColor': '#FDB863',
        'color': '#D95F02',   # dunklere Randfarbe
        'weight': 2.5,        # dickere Linie
        'fillOpacity': 0.15,  # etwas transparenter
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["bezeichnun", "mietkatego"],
        aliases=["Quartier", "Mietkategorie"],
        localize=True,
        sticky=True
    )
).add_to(m)

# KMeans-Cluster einfügen (zum Vergleich mit spatial-Ergebnis)
# cluster_kmeans_layer = folium.FeatureGroup(name="Cluster (kMeans)", show=True)
# for _, r in gdf.iterrows():
#     c = cluster_colors.get(r["cluster_kmeans"], "#666666")
#     folium.CircleMarker(
#         location=[r.lat, r.lon],
#         radius=3,
#         color=c,
#         fill=True,
#         fill_color=c,
#         fill_opacity=0.85,
#         tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r["cluster_kmeans"]}"
#     ).add_to(cluster_kmeans_layer)
# cluster_kmeans_layer.add_to(m)

# Clusterpunkte layer
cluster_skater_layer = folium.FeatureGroup(name="Cluster (SKATER)", show=True)
for _, r in gdf.iterrows():
    c = cluster_colors.get(r["cluster_skater"], "#666666")
    folium.CircleMarker(
        location=[r.lat, r.lon],
        radius=3,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.85,
       tooltip=f"{r.Straßenname} {r.Hsnr} – Cluster {r["cluster_skater"]}"
    ).add_to(cluster_skater_layer)

cluster_skater_layer.add_to(m)
folium.LayerControl().add_to(m)

# export map to html
html_path = f"maps/brb_{NUMBER_OF_CLUSTERS}_clusters.html"
m.save(html_path)
#m

In [ ]:
# plot histogram of form [(np.int64(0), np.int64(3)), (np.int64(1), np.int64(4)), (np.int64(2), np.int64...
import matplotlib.pyplot as plt
cluster_counts = gdf["cluster_skater"].value_counts().sort_index()
plt.figure(figsize=(10,6))
plt.bar(cluster_counts.index.astype(str), cluster_counts.values, color='skyblue')
plt.xlabel('Cluster')
plt.ylabel('Anzahl Adressen')
plt.title('Anzahl der Adressen pro Cluster (SKATER)')
plt.xticks(cluster_counts.index.astype(str))
plt.grid(axis='y')
plt.show()

# Abgleich mit Ortsteilen, Quartieren und Mietkategorien
Zum Abgleich der ermittelten Wohnlagen mit bestehenden Strukturen werden die Cluster mit den Quartieren und Ortsteilen der Adressen sowie den Mietkategorien (sofern vorhanden) verglichen. In der Kreuztabelle können die Verteilungen der Cluster über die verschiedenen räumlichen Einheiten analysiert werden.

In [ ]:
# Kreuztabelle zu Quartieren
ct = pd.crosstab(gdf["quartier"], gdf["cluster_skater"], normalize="index")
plt.figure(figsize=(14,10))
sns.heatmap(ct, cmap="viridis", annot=False)
plt.title("Quartier vs. SKATER-Cluster (Zeilen normalize: Anteil pro Quartier)")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Quartier")
plt.tight_layout()
plt.show()

Die Darstellung zeigt, wie sich die automatisch gebildeten SKATER-Cluster auf die bestehenden Wohnquartiere verteilen. Deutlich wird, dass mehrere Quartiere eine klare Dominanz einzelner Cluster aufweisen (z. B. Zentrum, Nord, Görden oder Hohenstücken), was auf eine vergleichsweise homogene interne Struktur schließen lässt. Andere Quartiere, insbesondere jene mit gemischten Nutzungen oder großen räumlichen Ausdehnungen, verteilen sich stärker auf mehrere Cluster. Diese Heterogenität ist erwartbar und spiegelt eher die interne Vielfalt der Quartiere wider als Schwächen im Clustering. Insgesamt zeigt die Heatmap, dass die Clusterbildung bestehende räumliche Zusammenhänge gut trifft.

In [ ]:
# Kreuztabelle zu Ortsteilen
ct = pd.crosstab(gdf["ortsteil"], gdf["cluster_skater"], normalize="index")

plt.figure(figsize=(12,8))
sns.heatmap(ct, cmap="magma", annot=False)
plt.title("Ortsteil vs. SKATER-Cluster")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Ortsteil")
plt.tight_layout()
plt.show()

Die Zuordnung der SKATER-Cluster zu den Ortsteilen zeigt überwiegend klare Muster. Viele Ortsteile werden überwiegend einem oder zwei Clustern zugeordnet, was auf eine konsistente und funktional nachvollziehbare Lagecharakteristik schließen lässt. Dies ist besonders bei peripheren oder dörflichen Ortsteilen sichtbar, die sich typischerweise durch ähnliche Infrastrukturausstattung und Distanzlagen auszeichnen. Gleichzeitig treten - abhängig von Größe und Struktur des Ortsteils – einzelne Streuungen auf, die auf interne Unterschiede oder Übergangsbereiche hindeuten können. Insgesamt bestätigt die Darstellung, dass die Cluster auch außerhalb des Kernstadtgebiets sinnvolle, zusammenhängende Lagemuster abbilden.

In [ ]:
ct = pd.crosstab(gdf["mietkategorie"], gdf["cluster_skater"], normalize="columns")

plt.figure(figsize=(10,6))
sns.heatmap(ct, cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Mietkategorie vs. SKATER-Cluster (Spalten normalize)")
plt.xlabel("SKATER-Cluster")
plt.ylabel("Mietkategorie")
plt.tight_layout()
plt.show()


Die Kreuztabelle zwischen Mietkategorien und SKATER-Clustern dient der ökonomischen Validierung des Modells. Hier zeigt sich, dass bestimmte Cluster deutlich mit niedrigeren, mittleren oder höheren Mietkategorien assoziiert sind. Mehrere Cluster weisen eine hohe Übereinstimmung mit spezifischen Mietniveaus auf, was darauf hinweist, dass die algorithmisch erkannten Lagegruppen auch sozioökonomische Unterschiede in der Wohnlagequalität widerspiegeln. Streuungen in einzelnen Kategorien sind zu erwarten und spiegeln natürliche Übergänge oder heterogene Straßenzüge wider. Insgesamt deutet die Struktur jedoch auf eine plausibel differenzierende Wirkung der Cluster hin.

In [ ]:
# Alle relevanten Z-Variablen für die Clusterinterpretation
z_vars = [col for col in gdf.columns if col.startswith("z_")]

cluster_profile = gdf.groupby("cluster_skater")[z_vars].mean()

plt.figure(figsize=(18,10))
sns.heatmap(cluster_profile, cmap="coolwarm", center=0, annot=False)
plt.title("Feature-Profil der SKATER-Cluster (alle Z-Scores)")
plt.xlabel("Feature")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()


## Abgleich mit Gebäudetypologie
Die Gebäudetypologie wurde nicht in die Clusterbildung einbezogen, da sie primär die Nutzung einzelner Gebäude beschreibt und keine geeignete räumlich-strukturelle Größe zur Abgrenzung von Wohnlagen darstellt. Stattdessen dient sie hier der Interpretation und Plausibilisierung der identifizierten Wohnlagen-Cluster.

In [ ]:
# building_type_cols = deine One-Hot-Spalten
bt_long = (
    gdf[["cluster_skater"] + building_type_cols]
    .set_index("cluster_skater")
    .stack()
    .reset_index()
)
bt_long.columns = ["cluster_skater", "building_type", "value"]

# nur echte Zuordnungen
bt_long = bt_long[bt_long["value"] == 1]
ct = pd.crosstab(
    bt_long["building_type"],
    bt_long["cluster_skater"],
    normalize="columns"
)

plt.figure(figsize=(12, 7))
sns.heatmap(
    ct,
    cmap="coolwarm",
    annot=True,
    fmt=".2f"
)

plt.title("Gebäudetyp vs. Cluster (Spalten-normalisiert)")
plt.xlabel("Cluster")
plt.ylabel("Gebäudetyp")
plt.tight_layout()
plt.show()

# Export-Pipeline

In [ ]:
# Save GeoDataFrame with scores and clusters as CSV
import os

# -------------------------------------------
# 0) EXPORT-PFAD
# -------------------------------------------
EXPORT_PATH = "out"
EXPORT_GPKG = os.path.join(EXPORT_PATH, "wohnlagen_brb.gpkg")
EXPORT_CSV = os.path.join(EXPORT_PATH, "wohnlagen_brb.csv")

# Ordner anlegen
os.makedirs(EXPORT_PATH, exist_ok=True)

# Falls alte Datei existiert: löschen (sonst doppelte Layer)
if os.path.exists(EXPORT_GPKG):
    os.remove(EXPORT_GPKG)
    print(f"Alte Datei gelöscht: {EXPORT_GPKG}")

# -------------------------------------------
# 1) HILFSFUNKTION: Sicheres Schreiben
# -------------------------------------------

def write_layer(gdf, layer_name, crs=EPSG_4326):
    """
    Schreibt einen GeoDataFrame als Layer in die GeoPackage-Datei.
    Stellt sicher, dass das CRS korrekt ist.
    """
    if gdf is None or len(gdf) == 0:
        print(f"⚠ Layer '{layer_name}' übersprungen: leer oder None")
        return

    # CRS prüfen
    if gdf.crs is None:
        print(f"⚠ GDF '{layer_name}' hat kein CRS – setze auf {crs}")
        gdf = gdf.set_crs(crs)
    elif gdf.crs.to_string() != crs:
        print(f"🔄 Reprojiziere '{layer_name}' nach {crs}")
        gdf = gdf.to_crs(crs)

    # Schreiben
    gdf.to_file(EXPORT_GPKG, layer=layer_name, driver="GPKG")
    print(f"✔ Exportiert: {layer_name}  →  {EXPORT_GPKG}")


# -------------------------------------------
# 2) LAYER DEFINIEREN
# -------------------------------------------
layers = {
    "wohnadressen": gdf,
    "wohnlagen_score": gdf[[col for col in gdf.columns if col.startswith("score_") or col in ["geometry"]]],
    #"cluster_skater": gdf[["cluster_skater", "geometry"]] if "cluster_skater" in gdf.columns else None,
    "cluster_kmeans": gdf[["cluster_kmeans", "geometry"]] if "cluster_kmeans" in gdf.columns else None,
    "lageklassen": gdf[["lageklasse", "score_total", "geometry"]] if "lageklasse" in gdf.columns else None,

    # Infrastruktur-Daten
    "bahnlinien": rails,
    #"laerm_lden": gdf_laerm,                      # falls du gdf_laerm heißt
    "haltestellen": gdf_haltestellen if 'gdf_haltestellen' in globals() else None,
    #"medzentren": gdf_medzentren if 'gdf_medzentren' in globals() else None,
    #"einzelhandel": gdf_einzelhandel if 'gdf_einzelhandel' in globals() else None,

    # Debug-Daten
    "bahnschnitt_debug": gdf[["behind_rail_from_center", "center_route", "geometry"]] if "behind_rail_from_center" in gdf.columns else None,
}

# -------------------------------------------
# 3) EXPORT AUSFÜHREN
# -------------------------------------------

print("🔁 Exportiere alle Layer …\n")

for layer_name, layer_gdf in layers.items():
    write_layer(layer_gdf, layer_name)

print("\n🎉 Fertig! Die GeoPackage-Datei liegt hier:")
print(f"📦 {EXPORT_GPKG}")
gdf_csv = gdf.drop(columns=['geometry'], errors='ignore')
gdf_csv.to_csv(EXPORT_CSV, index=False, encoding='utf-8')
print(f"\U0001f4c4 CSV exportiert: {EXPORT_CSV}")